In [2]:
#
from glob import glob as gg
#
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import rc as rc
#
from scipy.io import loadmat
#
from scipy.interpolate import interp1d
#
from netCDF4 import Dataset  
#
from datetime import datetime
from datetime import date
#
import pandas as pd
#
import gsw

In [3]:
#
# load time properties
#
class REcoM_time:
    def __init__(self):
        # load time properties
        self.estimate_time_axis()
    def estimate_time_axis(self):
        path_obs = '/home/fbirrien/NuArctic/nuarctic/REcoM1D/config/'
        filename = path_obs + 'time.recom'
        # read data 
        with open(filename) as f:
            lines = f.readlines()
        for l in lines:
            tmp = l.replace(' ','').replace('\t','').split('=')[-1].strip()
            if 'start_date' in l:
                date_start = datetime.strptime(tmp,'%d-%m-%Y').toordinal()
            elif 'end_date' in l:
                date_end = datetime.strptime(tmp,'%d-%m-%Y').toordinal()
            elif 'dt' in l:
                dt = float(tmp)/24.
            elif 'spinup' in l:
                spinup = int(tmp)
        self.date_start, self.date_end = np.copy(date_start), np.copy(date_end)
        date_start = date_start - spinup
        
        # create time axis
        npt = int(abs(date_end - date_start)/dt)
        self.dates = np.linspace(date_start, date_end, npt+1)        
#
time = REcoM_time()

In [10]:
# load ITP_ocean data
class sensor:
    def __init__(self, dates, longitude, latitude, pressure, temperature, salinity):
        self.dates, self.longitude, self.latitude = dates, longitude, latitude
        self.pressure, self.temperature, self.salinity = pressure, temperature, salinity
        # compute depth from pressure and latitude
        self.compute_depth()
        
    def compute_depth(self):
        
        depth = []
        for lat in self.latitude:
            tmp = gsw.z_from_p(self.pressure, lat)
            depth.append(gsw.z_from_p(self.pressure, lat))
        self.depth = np.asarray(depth)
            
class ocean_data:
    def __init__(self):
        self.read_ocean_data()
        
    def read_ocean_data(self):
        # load .mat files
        path_obs = '/home/fbirrien/Data/Observations/MOSAiC/ocean/'
        filename = path_obs + 'mosaic.whoiitpmerged.newgrid.mat'
        matdata = loadmat(filename)
        
        # dates (time axis)
        dates = matdata['julds']-366.
        
        # coordinates
        longitude, latitude = matdata['lons'], matdata['lats']
        
        # pressure (depth axis)
        pressure = matdata['pressg'][0]
        
        # salinity amd temperature
        salinity, temperature = matdata['salg'],matdata['ptmpg']

        # sensor 1: ITP92 
        sensor1 = sensor(dates[:,0], longitude[:,0], latitude[:,0], pressure, temperature[:,0], salinity[:,0])
        # sensor 2: ITP101
        sensor2 = sensor(dates[:,1], longitude[:,1], latitude[:,1], pressure, temperature[:,1], salinity[:,1])
        # sensor 3: ITP112
        sensor3 = sensor(dates[:,-1], longitude[:,-1], latitude[:,-1], pressure, temperature[:,-1], salinity[:,-1])
        
        # store data
        self.sensor1, self.sensor2, self.sensor3 = sensor1, sensor2, sensor3

#

ITP_data =  ocean_data()